### **Internet search Agent**

In [9]:
import os
from dotenv import load_dotenv 
from langchain_openai import ChatOpenAI 
from langchain_core.tools import tool 
import requests 
from langchain_community.tools import DuckDuckGoSearchRun
load_dotenv()

True

In [21]:
llm = ChatOpenAI() 

In [68]:
search_tool = DuckDuckGoSearchRun() # Deprecated

In [54]:
from ddgs import DDGS

ddgs = DDGS()
results = ddgs.text(query="Russia latest news related to war", max_results = 3)


with DDGS() as ddgs:
    results = ddgs.text(query="Current news related to war", max_results = 3)
    for result in results:
        print(result["title"])
        print(result["body"])
        print(result["href"])
        print("\n================\n")

War News Today USA: A Current Update on the Latest Developments - Football Formation
Warnews is more than just headlines; it's a lens through which we understand the world's most pressing issues. Whether it's domestic conflict or in ... WarNews Today USA: A Current Update on the Latest Developments ... From the battlefield to the media landscape, it’s clear that war will remain a central issue in global and national discourse. Understanding the impact of warnews, both domestic and international, is essential for shaping future policies and responses to conflict.
https://football-formation.com/war-news-today-usa-a-current-update-on-the-latest-developments/


Ground News
Stay current with all the latest and breaking news about War , compare headlines and perspectives between news sources on stories happening today. In total, 3,159 stories have been published about War which Ground News has aggregated in the past 3 months. ... MOSCOW (Reuters) – The Kremlin on Wednesday criticised comment

#### **Creating custom search tool using DDGS**

In [69]:
@tool 
def search_web(query: str) -> str:
    """
    Search the web for current information using DDGS.
    Use this tool when you need to find recent information about any topic.
    """
    with DDGS() as ddgs:
        try:
            results = ddgs.text(query = query, max_results = 3)
            formated_query = []
            for result in results:
                formated_query.append(f"Title: {result['title']}\nContent: {result['body']}\nURL: {result['href']}")
                return "\n\n".join(formated_query)
        except Exception as e:
            return f"Error Searching: {str(e)}"

In [70]:
search_tool = search_web

In [72]:
print(search_tool.invoke("What is best source of income"))

Title: 25 Passive Income Ideas To Make Extra Money In 2025 | Bankrate
Content: Passive income can be a great way to generate extra cash flow and build financial security. Here are 25 passive income ideas to get you started.
URL: https://www.bankrate.com/investing/passive-income-ideas/


In [73]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

In [74]:
# Pulling langchain React prompt 
prompt = hub.pull("hwchase17/react")

c:\Users\hites\OneDrive\Desktop\Langchain\venv\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [75]:
# Create react agent manually with pulled prompt
agent = create_react_agent(
    llm=llm, 
    tools = [search_tool], 
    prompt = prompt
)

In [76]:
# Wrap it with AgentExecutor
agent_executor = AgentExecutor(
    agent=agent, 
    tools = [search_tool], 
    verbose=True
)

In [77]:
# Invoke
response = agent_executor.invoke({"input": "Best way to reach new york from mumbai"})
print(response)



> Entering new AgentExecutor chain...
It would be best to search online for the most current and efficient way to reach New York from Mumbai.
Action: search_web
Action Input: "best way to reach New York from Mumbai"Title: Mumbai to Newyork - 8 ways to travel via train, plane, and ...
Content: What is the fastest way to get from Mumbai to Newyork? The fastest way to get from Mumbai to Newyork is to fly and bus which takes 19h 11m and costs £310 - £550 ...
URL: https://www.rome2rio.com/s/Mumbai/NewyorkBased on the search results, flying and taking a bus seems to be the fastest way to reach New York from Mumbai.
Final Answer: The best way to reach New York from Mumbai is to fly and take a bus.

> Finished chain.
{'input': 'Best way to reach new york from mumbai', 'output': 'The best way to reach New York from Mumbai is to fly and take a bus.'}


In [78]:
response['output']

'The best way to reach New York from Mumbai is to fly and take a bus.'

In [79]:
response["input"]

'Best way to reach new york from mumbai'

#### **Weather Agent**

In [81]:
import os
import requests

@tool 
def get_weather(city: str) -> str:
    """
    This function fetches the current weather data for a given city
    """
    url = f'https://api.weatherstack.com/current?access_key=096476fdc03f5b6daff559ed0a676e31&query={city}'
    response = requests.get(url)

    return response.json()

In [83]:
llm = ChatOpenAI()

In [84]:
# Prompt from hub 
prompt = hub.pull("hwchase17/react")

c:\Users\hites\OneDrive\Desktop\Langchain\venv\lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [87]:
# Create React agent manually using pulled prompt
agent = create_react_agent(
    llm=llm, 
    prompt=prompt, 
    tools=[search_tool, get_weather], 
)

In [88]:
# Agent Executor wrapper 
agent_executor = AgentExecutor(
    agent=agent, 
    tools = [search_tool, get_weather], 
    verbose=True
)

In [91]:
response = agent_executor.invoke({"input": "Find the Capital of Maharashtra, then find it's current weather condition"})



> Entering new AgentExecutor chain...
I should search for the capital of Maharashtra and then get the current weather data for that city.
Action: search_web
Action Input: Capital of MaharashtraTitle: Capital of Maharashtra
Content: Capital of Maharashtra may refer to: Mumbai, summer capital of Maharashtra - Largest City in Maharashtra Nagpur, winter capital of Maharashtra - Third largest city in Maharashtra
URL: https://en.wikipedia.org/wiki/Capital_of_MaharashtraI now need to check the current weather condition for Mumbai and Nagpur
Action: get_weather
Action Input: Mumbai{'request': {'type': 'City', 'query': 'Mumbai, India', 'language': 'en', 'unit': 'm'}, 'location': {'name': 'Mumbai', 'country': 'India', 'region': 'Maharashtra', 'lat': '18.975', 'lon': '72.826', 'timezone_id': 'Asia/Kolkata', 'localtime': '2025-07-20 10:11', 'localtime_epoch': 1753006260, 'utc_offset': '5.50'}, 'current': {'observation_time': '04:41 AM', 'temperature': 29, 'weather_code': 143, 'weather_icons': ['

In [92]:
response["output"]

'The capital of Maharashtra is Mumbai, and the current weather condition there is Haze with a temperature of 29°C.'